In [134]:
import pandas as pd
import matplotlib.pyplot as plt
from transformers import pipeline
import torch
import os
from huggingface_hub import hf_hub_download

In [ ]:
HUGGING_FACE_API_KEY = os.environ.get("HUGGING_FACE_API_KEY")
model_id_1 = "nlptown/bert-base-multilingual-uncased-sentiment"
model_id_2 = "cardiffnlp/twitter-roberta-base-sentiment"
filenames = [
    "pytorch_model.bin", "added_tokens.json", "config.json", "generation_config.json",
    "special_tokens_map.json", "spiece.model", "tokenizer_config.json"
]

In [137]:
reviews = pd.read_csv('./rawdata/cleaned_reviews.csv')
summary = pd.read_csv('./rawdata/asin_summary.csv')

In [138]:
reviews.head()

,rating,title_x,text,asin,timestamp,helpful_vote,title_y,features,price
0,5,Fast!,I have been using laptops for the past 30 year...,B089HR6CQP,1.601470e+12,0,"FusionTech Gaming G3 15 3500, 15.6 inch FHD La...",[15.6 inch FHD (1920 x 1080) 120Hz 250 nits WV...,799.0
1,5,Works Great - No Problems - Great Value for Price,"I read a bunch of negative reviews, so I wante...",B089HR6CQP,1.626710e+12,0,"FusionTech Gaming G3 15 3500, 15.6 inch FHD La...",[15.6 inch FHD (1920 x 1080) 120Hz 250 nits WV...,799.0
2,5,Excellent gaming computer for the price,"Runs smooth, fast, and works well for gaming. ...",B089HR6CQP,1.614820e+12,0,"FusionTech Gaming G3 15 3500, 15.6 inch FHD La...",[15.6 inch FHD (1920 x 1080) 120Hz 250 nits WV...,799.0
3,1,Overheating on setup,I'm impressed. I've never seen a computer so w...,B089HR6CQP,1.626820e+12,0,"FusionTech Gaming G3 15 3500, 15.6 inch FHD La...",[15.6 inch FHD (1920 x 1080) 120Hz 250 nits WV...,799.0
4,1,Don’t buy,"I bought 2 of these laptops, asked for replace...",B089HR6CQP,1.611410e+12,22,"FusionTech Gaming G3 15 3500, 15.6 inch FHD La...",[15.6 inch FHD (1920 x 1080) 120Hz 250 nits WV...,799.0


In [139]:
summary.head()

,asin,avg_rating,num_reviews,price,title_y,features,os,color
0,B004PANKIA,5.000000,1,NaN,FusionTech ABC 15 Laptop - High-Octane Enterta...,"[Genuine Windows 7 Home Premium, 64bit, Intel ...",Windows 7;,Silver
1,B0052F35I2,3.750000,12,NaN,"FusionTech ABC 17 Laptop, i7-2630QM, 8GB DDR3 ...","[Intel Core i7 Processor 2GHz, 6GB DDR3 RAM, 6...",Windows 7,NaN
2,B0057CAGUA,5.000000,1,NaN,FusionTech Worktop 17R i17R Core i5-2430M 2.4G...,[],Microsoft Windows 7 Home Premium (64 bit),Red
3,B005SDDXF8,4.333333,6,NaN,"FusionTech ABC 15 L502X Intel Core i5-2410, 2....",[2nd generation Intel Core i5-2 410M processor...,Windows 7 Home Premium (64-bit),NaN
4,B0081YPX3Q,2.642857,14,NaN,FusionTech ABC15-9375sLV 15-Inch Laptop (2.1 G...,"[Intel Core i7 3612QM Processor 2.1GHz, 8 GB D...",Windows 7,Silver


In [140]:
print(summary.columns)
print(reviews.columns)

Index(['asin', 'avg_rating', 'num_reviews', 'price', 'title_y', 'features',
       'os', 'color'],
      dtype='object')
Index(['rating', 'title_x', 'text', 'asin', 'timestamp', 'helpful_vote',
       'title_y', 'features', 'price'],
      dtype='object')


In [141]:
reviews.head()

,rating,title_x,text,asin,timestamp,helpful_vote,title_y,features,price
0,5,Fast!,I have been using laptops for the past 30 year...,B089HR6CQP,1.601470e+12,0,"FusionTech Gaming G3 15 3500, 15.6 inch FHD La...",[15.6 inch FHD (1920 x 1080) 120Hz 250 nits WV...,799.0
1,5,Works Great - No Problems - Great Value for Price,"I read a bunch of negative reviews, so I wante...",B089HR6CQP,1.626710e+12,0,"FusionTech Gaming G3 15 3500, 15.6 inch FHD La...",[15.6 inch FHD (1920 x 1080) 120Hz 250 nits WV...,799.0
2,5,Excellent gaming computer for the price,"Runs smooth, fast, and works well for gaming. ...",B089HR6CQP,1.614820e+12,0,"FusionTech Gaming G3 15 3500, 15.6 inch FHD La...",[15.6 inch FHD (1920 x 1080) 120Hz 250 nits WV...,799.0
3,1,Overheating on setup,I'm impressed. I've never seen a computer so w...,B089HR6CQP,1.626820e+12,0,"FusionTech Gaming G3 15 3500, 15.6 inch FHD La...",[15.6 inch FHD (1920 x 1080) 120Hz 250 nits WV...,799.0
4,1,Don’t buy,"I bought 2 of these laptops, asked for replace...",B089HR6CQP,1.611410e+12,22,"FusionTech Gaming G3 15 3500, 15.6 inch FHD La...",[15.6 inch FHD (1920 x 1080) 120Hz 250 nits WV...,799.0


In [142]:
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')

PyTorch version: 2.5.1
CUDA available: False


In [143]:
# Load pre-trained sentiment analysis pipelines for multiple models
sentiment_pipeline_bert = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
sentiment_pipeline_roberta = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

# Create DataFrame
reviews = pd.DataFrame(data)

# Function to analyze sentiment using Hugging Face model
def analyze_sentiment(text, pipeline):
    result = pipeline(text)[0]  # Get the first result (Hugging Face returns a list)
    if result['label'] in ['POSITIVE', '5 stars']:  # Adjust labels based on model
        return result['score']  # Return positive score (0 to 1)
    elif result['label'] in ['NEGATIVE', '1 star']:
        return -result['score']  # Return negative score (-1 to 0)
    else:
        return 0  # Neutral sentiment

# Apply sentiment analysis using both BERT and RoBERTa pipelines
reviews['bert_sentiment'] = reviews['text'].apply(lambda x: analyze_sentiment(x, sentiment_pipeline_bert))
reviews['roberta_sentiment'] = reviews['text'].apply(lambda x: analyze_sentiment(x, sentiment_pipeline_roberta))

# Plotting the results for comparison
plt.figure(figsize=(10, 6))
x = range(len(reviews))  # Index for each text entry

plt.plot(x, reviews['bert_sentiment'], label='BERT Sentiment', marker='o', linestyle='-')
plt.plot(x, reviews['roberta_sentiment'], label='RoBERTa Sentiment', marker='x', linestyle='--')

plt.title('Sentiment Analysis Comparison (BERT vs RoBERTa)')
plt.xlabel('Text Index')
plt.ylabel('Sentiment Score')
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)  # Neutral line
plt.legend()
plt.grid(True)
plt.xticks(x, [f'Text {i+1}' for i in x])  # Label x-axis with text indices
plt.show()

RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.